In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer 
# from sklearn.decomposition import TruncatedSVD
# from sklearn.decomposition import NMF
# from sklearn.metrics.pairwise import cosine_similarity


import numpy as np
from sklearn.manifold import TSNE

from sklearn.neighbors import NearestNeighbors

In [2]:
# Load data

df_4 = pd.read_csv("C:/Users/storm/Desktop/METIS/Project/Project 5_RS/Dataset/sephora_SG_FIREFOX_dataset_clean.csv")



In [3]:
df_4.head()

,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,cleanser-and-exfoliator,FRESH,Soy Face Cleanser,113,4.5,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1,1,1,1,1
1,cleanser-and-exfoliator,INDIE LEE,Brightening Cleanser,50,4.5,"Water (Aqua/Eau), Decyl Glucoside (Veg. Oils &...",1,1,1,1,1
2,cleanser-and-exfoliator,TARTE,Sea Deep Dive Makeup Removing Gel Cleanser,40,4.5,"Water/Aqua/Eau, cocamidopropyl betaine, disodi...",1,1,1,1,1
3,cleanser-and-exfoliator,FRESH,Rose Cleansing Foam • 120ml,47,4.0,"Aqua (Water), Butylene Glycol, Decyl Glucoside...",1,1,1,1,1
4,cleanser-and-exfoliator,DR. ROEBUCKS,Noosa Nourishing Crème Cleanser • 100ml,30,4.5,"Aqua/Water/Eau, Stearic Acid, Cetearyl Alcohol...",0,1,1,1,1


In [4]:
# All possible combinations for the option choices

option_1 = df_4.Label.unique().tolist()
option_2 = df_4.columns[6:].tolist()

In [5]:
# User input for product category & skin-type

op_1 = 'toner'
op_2 = 'Dry'

In [6]:
# Filter dataframe by given options

df = df_4[df_4['Label'] == op_1][df_4[op_2] == 1]
df = df.reset_index()

df.head()

C:\Users\storm\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,index,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,109,toner,PIXI,Glow Tonic,48,4.0,"Aqua, Aloe Barbadensis Leaf Juice, Hamamelis V...",1,1,1,1,0
1,110,toner,FRESH,Rose Deep Hydration Facial Toner • 250ml,69,4.0,"Aqua (Water), Glycerin, Butylene Glycol, Pheno...",1,1,1,1,1
2,111,toner,ORIGINS,Dr. Andrew Weil for Origins™ Mega-Mushroom Rel...,63,4.5,"Water\Aqua\Eau , Butylene Glycol , Peg-4 , Cit...",1,1,1,1,0
3,112,toner,FRESH,Black Tea Kombucha Facial Treatment Essence,108,4.5,"Aqua (Water), Saccharomyces Ferment Filtrate, ...",1,1,1,1,0
4,113,toner,CAUDALIE,Vinopure Clear Skin Purifying Toner • 200ml,35,4.0,"Water, Alcohol Denat.*, Rosa Damascena Flower ...",1,1,1,1,1


In [7]:
#Convert dataframe to series object to be able to parse into CountVectorizer

text = df_4['ingredients']
type(text)

pandas.core.series.Series

In [8]:
# instantiate the vectorizer object
#Tokenize by every comma
vectorizer = CountVectorizer(lowercase=True, tokenizer=lambda x: x.split(', '))

# convert the documents into a document-term matrix
doc_word = vectorizer.fit_transform(text)
doc_word.shape

(403, 2496)

In [9]:
# quick check on Tokens created by CountVec
tokens = vectorizer.get_feature_names()
tokens

['',
 ' gluconolactone.',
 "'+ naturally occurs in essential oils",
 '(evening primrose) oil',
 '* caudalie patented vine resveratrol: visibly resculpts and firms.\n* hyaluronic acid: plumps',
 '* haberlea leaf extract: supports the natural production of ingredients essential to its beauty.\n\n* french mallow: supports natural aquaporin synthesis to invigorate water circulation.',
 '*argania spinosa kernel oil',
 '*camellia sinensis leaf water(green tea)',
 '*olea europaea (olive) fruit oil',
 '*prunus amygdalus dulcis (sweet almond) oil',
 '*prunus amygdalus dulcis (sweet almond) oil,* argania spinosa kernel oil,* olea europaea (olive) fruit oil',
 '*tilia vulgaris flower extract',
 '- organic grape water: hydrates.\n- grape seed polyphenols: antioxidants.\n- vinolevure: strengthens and moisturizes.\n- flax seed powder: matifies.\nwater',
 '-organic grape water: moisturizes and decreases sensitivity. -organic chamomile extract: provides anti-redness properties and soothes the skin. -p

In [10]:
test = pd.DataFrame(doc_word.toarray(), index=df_4.name, columns=vectorizer.get_feature_names())
test.head()

,,gluconolactone.,'+ naturally occurs in essential oils,(evening primrose) oil,* caudalie patented vine resveratrol: visibly resculpts and firms.\n* hyaluronic acid: plumps,* haberlea leaf extract: supports the natural production of ingredients essential to its beauty.\n\n* french mallow: supports natural aquaporin synthesis to invigorate water circulation.,*argania spinosa kernel oil,*camellia sinensis leaf water(green tea),*olea europaea (olive) fruit oil,*prunus amygdalus dulcis (sweet almond) oil,...,zinc sulfate,zingiber aromaticus extract,zingiber officinale,zingiber officinale (ginger) root extract,zingiber officinale (ginger) root extract*,zingiber officinale (ginger) root extract. *100% natural fragrance,zingiber officinale (ginger) root oil,zingiber officinale (ginger) water,zingiber ofﬁcinale (ginger) root extract,ziziphus jujuba fruit extract
name,,,,,,,,,,,,,,,,,,,,,
Soy Face Cleanser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brightening Cleanser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sea Deep Dive Makeup Removing Gel Cleanser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rose Cleansing Foam • 120ml,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Noosa Nourishing Crème Cleanser • 100ml,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Dimension reduction with t-SNE

model = TSNE(n_components = 2, learning_rate = 200)
tsne_features = model.fit_transform(doc_word)

# Make X, Y columns 
df_4['X'] = tsne_features[:, 0]
df_4['Y'] = tsne_features[:, 1]

In [12]:
doc_word

<403x2496 sparse matrix of type '<class 'numpy.int64'>'
	with 8075 stored elements in Compressed Sparse Row format>

In [13]:
df_4.head()

,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y
0,cleanser-and-exfoliator,FRESH,Soy Face Cleanser,113,4.5,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1,1,1,1,1,2.717109,1.460364
1,cleanser-and-exfoliator,INDIE LEE,Brightening Cleanser,50,4.5,"Water (Aqua/Eau), Decyl Glucoside (Veg. Oils &...",1,1,1,1,1,-1.378974,0.836640
2,cleanser-and-exfoliator,TARTE,Sea Deep Dive Makeup Removing Gel Cleanser,40,4.5,"Water/Aqua/Eau, cocamidopropyl betaine, disodi...",1,1,1,1,1,52.243549,31.995144
3,cleanser-and-exfoliator,FRESH,Rose Cleansing Foam • 120ml,47,4.0,"Aqua (Water), Butylene Glycol, Decyl Glucoside...",1,1,1,1,1,3.631032,1.957896
4,cleanser-and-exfoliator,DR. ROEBUCKS,Noosa Nourishing Crème Cleanser • 100ml,30,4.5,"Aqua/Water/Eau, Stearic Acid, Cetearyl Alcohol...",0,1,1,1,1,1.257834,-0.873098


In [14]:
# Save the file
df_4.to_csv('test_4_TSNE.csv', encoding = 'utf-8-sig', index = False)

In [15]:
# Save the file
test.to_csv('test_matrix.csv', encoding = 'utf-8-sig', index = False)

In [16]:
#Cosine similarity

In [17]:
df = df.reset_index().drop('index', axis = 1)
df['dist'] = 0.0e

myItem = df[df.name.str.contains('Expression Rising Lotion')]
myItem

SyntaxError: invalid syntax (<ipython-input-17-19e10b638255>, line 2)

In [ ]:
# getting the array for myItem
P1 = np.array([myItem.X.values, myItem.Y.values]).reshape(1, -1)
P1

In [ ]:
# cosine similarities with other items

for i in range(len(df)):
    P2 = np.array([df['X'][i], df['Y'][i]]).reshape(-1, 1)
    df.dist[i] = (P1 * P2).sum() / (np.sqrt(np.sum(P1))*np.sqrt(np.sum(P2)))
    

In [ ]:
df = df.sort_values('dist')
df[['name', 'brand', 'dist']].head(5)

In [ ]:
df.head()

In [ ]:
# ## defining a function embedding ingredients and decomposition at once

# def my_recommender(op_1, op_2):
#     df = df_4[df_4['Label'] == op_1][df_4[op_2] == 1]
#     df = df.reset_index()

#     # embedding each ingredients
#     ingredient_idx = {}
#     corpus = []
#     idx = 0

#     for i in range(len(df)):
#         ingreds = df['ingredients'][i]
#         ingreds = ingreds.lower()
#         tokens = ingreds.split(', ')
#         corpus.append(tokens)
#         for ingredient in tokens:
#             if ingredient not in ingredient_idx:
#                 ingredient_idx[ingredient] = idx
#                 idx += 1

#     # Get the number of items and tokens
#     M = len(df)                 # The number of the items
#     N = len(ingredient_idx)     # The number of the ingredients

#     # Initialize a matrix of zeros
#     A = np.zeros(shape = (M, N))

#     # Define the oh_encoder function
#     def oh_encoder(tokens):
#         x = np.zeros(N)
#         for t in tokens:
#             # Get the index for each ingredient
#             idx = ingredient_idx[t]
#             # Put 1 at the corresponding indices
#             x[idx] = 1
#         return x

#     # Make a document-term matrix
#     i = 0
#     for tokens in corpus:
#         A[i, :] = oh_encoder(tokens)
#         i += 1

#     # Dimension reduction with t-SNE
#     model = TSNE(n_components = 2, learning_rate = 200)
#     tsne_features = model.fit_transform(A)

#     # Make X, Y columns
#     df['X'] = tsne_features[:, 0]
#     df['Y'] = tsne_features[:, 1]

#     return df
    

In [ ]:
# # Create the dataframe for all combinations

# df_all = pd.DataFrame()
# for op_1 in option_1:
#     for op_2 in option_2:
#             temp = my_recommender(op_1, op_2)
#             temp['Label'] = op_1 + '_' + op_2
#             df_all = pd.concat([df_all, temp])

In [ ]:
# # Save the file
# df_all.to_csv('sephora_SG_FIREFOX_dataset_TSNE.csv', encoding = 'utf-8-sig', index = False)